In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import joblib
import datetime

In [2]:
from sklearn.linear_model import SGDRegressor
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler

In [3]:
calendar_df = pd.read_csv('calendar.csv')
sales_df = pd.read_csv('sales_train_validation.csv')
prices_df = pd.read_csv('sell_prices.csv')

In [47]:
dataframe = pd.read_pickle('preprocessed.pickle')

MemoryError: 

In [31]:
dataframe = dataframe[dataframe['d']>1100]
dataframe = dataframe[dataframe['d']<1885]

In [33]:
le = LabelEncoder()
for column in dataframe.columns:
    dtype = dataframe[column].dtypes
    if dtype == 'category':
        print("Transforming", column)
        le.fit(dataframe[column])
        dataframe[column] = le.transform(dataframe[column])
dataframe.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,rolling_mean_7,rolling_median_7,rolling_mean_14,rolling_median_14,rolling_mean_28,rolling_median_28,rolling_mean_35,rolling_median_35,rolling_mean_60,rolling_median_60
33539000,14370,14370,14370,14370,14370,14370,1101,0,2014-02-02,11401,...,0.285645,0.0,0.214233,0.0,0.606934,0.0,0.600098,0.0,0.649902,0.0
33539001,14380,14380,14380,14380,14380,14380,1101,0,2014-02-02,11401,...,0.000000,0.0,0.000000,0.0,0.357178,0.0,0.285645,0.0,0.199951,0.0
33539002,14390,14390,14390,14390,14390,14390,1101,0,2014-02-02,11401,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
33539003,14400,14400,14400,14400,14400,14400,1101,2,2014-02-02,11401,...,2.285156,2.0,2.572266,2.0,2.464844,2.0,2.427734,2.0,2.333984,2.0
33539004,14410,14410,14410,14410,14410,14410,1101,1,2014-02-02,11401,...,0.428467,0.0,0.785645,0.0,0.893066,0.0,0.885742,1.0,0.833496,0.5


In [34]:
labels = dataframe['sales']
train = dataframe.drop(labels='sales', axis=1)

In [35]:
#dropping date for now cuz error
train = train.drop(labels='date', axis=1)

In [36]:
print(datetime.datetime.now())
lgb_reg = lgb.LGBMRegressor(num_leaves=125,learning_rate=0.075,n_jobs=-1)
lgb_reg.fit(train,labels)
print(datetime.datetime.now())

2022-11-18 14:43:00.670135
2022-11-18 14:44:32.457895


In [37]:
joblib.dump(lgb_reg, '13daysmode2_785_10_lgb.pkl')

['13daysmode2_785_10_lgb.pkl']

In [7]:
test = pd.read_pickle('preprocessed.pickle')

le = LabelEncoder()
for column in test.columns:
    dtype = test[column].dtypes
    if dtype == 'category':
        print("Transforming", column)
        le.fit(test[column])
        test[column] = le.transform(test[column])
labels1 = test['sales']        
test = test.drop(labels='date', axis=1)
test = test.drop(labels='sales', axis=1)
test = test[test['d']>1885]

Transforming id
Transforming item_id
Transforming dept_id
Transforming cat_id
Transforming store_id
Transforming state_id
Transforming weekday
Transforming event_name_1
Transforming event_type_1
Transforming event_name_2
Transforming event_type_2
Transforming snap_CA
Transforming snap_TX
Transforming snap_WI


In [8]:
lgb2 = joblib.load("13daysmode2_785_10_lgb.pkl")
#rf = pd.read_pickle('13daysmode2_785_10.pkl')
preds1 = pd.DataFrame()
#predict for day 1914
for i in range(1886,1914):
    preds1['d_'+str(i)] = lgb2.predict(test[test['d']==i])
#df['d_'+str(i)] =rf2.predict(test)
#df=rf.predict(train[train['d']==1801])
preds1.head(10)

,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0.907413,0.944669,0.872443,0.912668,0.983117,1.206609,1.150813,0.814646,0.794489,0.746563,...,0.925931,1.075924,1.255282,0.943848,0.995269,1.026440,0.954719,1.015314,1.231845,1.252941
1,0.200790,0.169464,0.169464,0.201752,0.217694,0.244423,0.316802,0.287534,0.240422,0.231608,...,0.249909,0.286191,0.277085,0.213311,0.180804,0.180804,0.225664,0.278267,0.360938,0.391292
2,0.252535,0.229492,0.221208,0.221208,0.244365,0.726160,0.666272,0.653115,0.860760,0.794664,...,0.539293,0.632271,0.443609,0.371838,0.324816,0.318526,0.318526,0.415250,0.506528,0.456889
3,1.681304,1.530555,1.530555,1.556661,1.678816,1.965021,2.555606,1.756392,1.678614,1.586080,...,1.815348,1.981076,2.034487,1.779475,1.499495,1.664044,1.792211,1.761312,2.545109,2.852558
4,0.848186,0.896124,0.873788,0.840020,0.855808,1.159663,1.107523,0.898631,0.866475,0.869853,...,1.142165,1.353021,1.364993,1.123783,1.088307,1.032669,0.997493,1.103898,1.300413,1.229505
5,0.998765,0.960707,1.049224,0.955411,1.001302,1.389949,1.315548,1.143252,0.996027,1.133058,...,1.104315,1.449420,1.257104,0.999491,1.006638,0.978167,1.067272,1.066873,1.343310,1.371078
6,0.394778,0.310289,0.289127,0.308690,0.329506,0.361460,0.439903,0.312574,0.273161,0.250847,...,0.513123,0.506238,0.466482,0.447521,0.390122,0.386622,0.415392,0.410617,0.432508,0.415121
7,9.458382,8.832256,9.042787,8.511134,9.503630,12.388146,10.523239,8.323479,9.029512,10.989370,...,9.142871,11.801727,9.055964,6.662834,7.232418,7.366410,6.402979,6.896141,9.551046,8.060732
8,0.314839,0.297965,0.254071,0.208155,0.251996,0.275316,0.281728,0.264792,0.225911,0.222686,...,0.426638,0.488473,0.488473,0.385277,0.274894,0.274894,0.274894,0.311607,0.354260,0.354260
9,0.645381,0.549018,0.583482,0.530762,0.605123,0.666141,0.666908,0.535539,0.549693,0.509509,...,0.497396,0.680936,0.883868,0.595491,0.559926,0.533319,0.508359,0.520272,0.576901,0.556592


In [9]:
#preds = pd.concat([valid_df[fixed_cols], preds],axis=1,sort=False)
#group-bys to group into 12-level hierarchical time series to calculate WRMSSE 
groupbys = ('for_all', 'state_id', 'store_id', 'cat_id', 'dept_id',['state_id', 'cat_id'],  
            ['state_id', 'dept_id'], ['store_id', 'cat_id'],['store_id', 'dept_id'], 'item_id', 
            ['item_id', 'state_id'], ['item_id', 'store_id'])
train_df = pd.concat([sales_df.loc[:,:'state_id'],sales_df.loc[:,'d_1069':]],axis=1,sort=False)
print(train_df.head(100))
valid_df = train_df.iloc[:,-28:].copy()
train_df = train_df.iloc[:,:-28]
print(train_df.head(100))
print(valid_df.head(100))
train_d_cols = [col for col in train_df.columns if col.startswith('d_')]
weight_cols = train_df.iloc[:,-28:].columns.tolist()
train_df['for_all'] = "all" #for level 1 aggregation
fixed_cols = [col for col in train_df.columns if not col.startswith('d_')]
valid_d_cols = [col for col in valid_df.columns if col.startswith('d_')]
if not all([col in valid_df.columns for col in fixed_cols]):
    valid_df = pd.concat([train_df[fixed_cols],valid_df],axis=1,sort=False)
#weights corresponding all the 12-level hierarchical transformed 42840 time-series
weight_df = compute_weights(train_df,valid_df,weight_cols,groupbys,fixed_cols)
#train data transformed from 30490 timeseries to 42840 hirerachichal time-series
train_42840_df = convert_to_42840(train_df, train_d_cols, groupbys)
#validation data transformed from 30490 timeseries to 42840 hirerachichal time-series
valid_42840_df = convert_to_42840(valid_df, valid_d_cols, groupbys)

                               id        item_id    dept_id   cat_id store_id  \
0   HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1   HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2   HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3   HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4   HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   
..                            ...            ...        ...      ...      ...   
95  HOBBIES_1_099_CA_1_validation  HOBBIES_1_099  HOBBIES_1  HOBBIES     CA_1   
96  HOBBIES_1_100_CA_1_validation  HOBBIES_1_100  HOBBIES_1  HOBBIES     CA_1   
97  HOBBIES_1_102_CA_1_validation  HOBBIES_1_102  HOBBIES_1  HOBBIES     CA_1   
98  HOBBIES_1_103_CA_1_validation  HOBBIES_1_103  HOBBIES_1  HOBBIES     CA_1   
99  HOBBIES_1_104_CA_1_validation  HOBBIES_1_104  HOBBIES_1  HOBBIES     CA_1   

   state_id  d_1069  d_1070

In [10]:

#WRMSSE
def convert_to_42840(df, cols, groupbys):
    series_gen = {}
    for i, grp in enumerate(groupbys):
        #grop by corresponding group and calculating aggregate sales of each day
        tmp = df.groupby(grp)[cols].sum()
        #storing the aggregate sale values of each corresponding group
        for j in range(len(tmp)):
            series_gen[gen_series_name(tmp.index[j])] = tmp.iloc[j].values
    return pd.DataFrame(series_gen).T #creating a dataframe of each corresponding group and aggregate sales each day i.e., transformed into 42840 sales

#this method return the name of each group
def gen_series_name(name):
    if isinstance(name, str) | isinstance(name, int):
        return str(name)
    else:
        return "__".join(name)

#here we are computing weights using the last 28 day sales of train data and their prices
def compute_weights(train_df,valid_df,weight_cols,groupbys,fix_cols):
    weights_map = {}
    weight_df = train_df[["item_id", "store_id"] + weight_cols]
    weight_df = pd.melt(weight_df,id_vars=["item_id", "store_id"],var_name='d',value_name='sales')
    weight_df = weight_df.merge(calendar_df[['wm_yr_wk','d']], on='d', how='left')
    weight_df = weight_df.merge(prices_df, how="left", on=["item_id", "store_id", "wm_yr_wk"])
    #computing dollar sales 
    weight_df["dollar_sales"] = weight_df["sales"] * weight_df["sell_price"]
    weight_df = weight_df.set_index(["item_id", "store_id", "d"]).unstack(level=2)["dollar_sales"]
    weight_df = weight_df.loc[zip(train_df.item_id, train_df.store_id), :].reset_index(drop=True)
    weight_df = pd.concat([train_df[fix_cols], weight_df],
                          axis=1, sort=False)
    #computing the weights for each group keys
    for i,grp in enumerate(groupbys):
        ser_weight = weight_df.groupby(grp)[weight_cols].sum().sum(axis=1)
        ser_weight = ser_weight / ser_weight.sum()
        for j in range(len(ser_weight)):
            weights_map[gen_series_name(ser_weight.index[j])] = np.array([ser_weight.iloc[j]])
    weights = pd.DataFrame(weights_map).T / len(groupbys) #creating a dataframe with weights corresponding to each group keys of 42840 hierachical time-series
    return weights

#here we compute the rmsse using the true values and predicted values along with train data which is being used to scale
#train data is used to scale the squared-error as taking the consecutive difference of each day sales
def compute_rmsse(train_df, valid_df, pred_df):
    scale_lst = []
    for i in range(len(train_df)):
        val = train_df.iloc[i].values
        # to consider the periods following the first non-zero demand observed for the series under evaluation.
        val = val[np.argmax(val!=0):]
        #to scale the squared-error as taking the consecutive difference of each day sales
        scale = ((val[1:] - val[:-1]) ** 2).mean()
        #storing the scale value corresponding to each time series
        scale_lst.append(scale)
    scale_arr = np.array(scale_lst)
    #computing mean squared error
    num = ((pred_df - valid_df)**2).mean(axis=1)
    #scaled error i.e., root mean squared scaled error
    rmsse = (num/scale_arr).map(np.sqrt)
    return rmsse

#here we return the final score value i.e., WRMSSE
def custom_metric(train_df, valid_df, pred_df, weights):
    #obtaing RMSSE by calling compute_rmsse function
    rmsse = compute_rmsse(train_df, valid_df, pred_df)
    #WRMSSE of each 42840 time-series is computed as product of corresponding weights and RMSSE respectively
    ser_metric = pd.concat([weights, rmsse], axis=1, sort=False).prod(axis=1)
    return np.sum(ser_metric) #aggregation of each WRMSSE of 42840 time-series to get the final WRMSSE score

In [11]:
valid_df.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,for_all,d_1886,d_1887,d_1888,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,all,1,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,all,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,all,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,all,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,all,1,0,4,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,all,0,1,3,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,all,1,2,0,...,0,0,0,1,0,1,0,0,1,1
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,all,0,11,5,...,0,0,1,37,3,4,6,3,2,1
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,all,0,2,0,...,0,0,1,1,6,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,all,0,1,1,...,1,0,0,0,0,0,0,2,0,2


In [13]:
preds1.head(10)

,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0.907413,0.944669,0.872443,0.912668,0.983117,1.206609,1.150813,0.814646,0.794489,0.746563,...,0.925931,1.075924,1.255282,0.943848,0.995269,1.026440,0.954719,1.015314,1.231845,1.252941
1,0.200790,0.169464,0.169464,0.201752,0.217694,0.244423,0.316802,0.287534,0.240422,0.231608,...,0.249909,0.286191,0.277085,0.213311,0.180804,0.180804,0.225664,0.278267,0.360938,0.391292
2,0.252535,0.229492,0.221208,0.221208,0.244365,0.726160,0.666272,0.653115,0.860760,0.794664,...,0.539293,0.632271,0.443609,0.371838,0.324816,0.318526,0.318526,0.415250,0.506528,0.456889
3,1.681304,1.530555,1.530555,1.556661,1.678816,1.965021,2.555606,1.756392,1.678614,1.586080,...,1.815348,1.981076,2.034487,1.779475,1.499495,1.664044,1.792211,1.761312,2.545109,2.852558
4,0.848186,0.896124,0.873788,0.840020,0.855808,1.159663,1.107523,0.898631,0.866475,0.869853,...,1.142165,1.353021,1.364993,1.123783,1.088307,1.032669,0.997493,1.103898,1.300413,1.229505
5,0.998765,0.960707,1.049224,0.955411,1.001302,1.389949,1.315548,1.143252,0.996027,1.133058,...,1.104315,1.449420,1.257104,0.999491,1.006638,0.978167,1.067272,1.066873,1.343310,1.371078
6,0.394778,0.310289,0.289127,0.308690,0.329506,0.361460,0.439903,0.312574,0.273161,0.250847,...,0.513123,0.506238,0.466482,0.447521,0.390122,0.386622,0.415392,0.410617,0.432508,0.415121
7,9.458382,8.832256,9.042787,8.511134,9.503630,12.388146,10.523239,8.323479,9.029512,10.989370,...,9.142871,11.801727,9.055964,6.662834,7.232418,7.366410,6.402979,6.896141,9.551046,8.060732
8,0.314839,0.297965,0.254071,0.208155,0.251996,0.275316,0.281728,0.264792,0.225911,0.222686,...,0.426638,0.488473,0.488473,0.385277,0.274894,0.274894,0.274894,0.311607,0.354260,0.354260
9,0.645381,0.549018,0.583482,0.530762,0.605123,0.666141,0.666908,0.535539,0.549693,0.509509,...,0.497396,0.680936,0.883868,0.595491,0.559926,0.533319,0.508359,0.520272,0.576901,0.556592


In [14]:
preds1 = pd.concat([valid_df[fixed_cols], preds1],axis=1,sort=False)
valid_d_cols

['d_1886',
 'd_1887',
 'd_1888',
 'd_1889',
 'd_1890',
 'd_1891',
 'd_1892',
 'd_1893',
 'd_1894',
 'd_1895',
 'd_1896',
 'd_1897',
 'd_1898',
 'd_1899',
 'd_1900',
 'd_1901',
 'd_1902',
 'd_1903',
 'd_1904',
 'd_1905',
 'd_1906',
 'd_1907',
 'd_1908',
 'd_1909',
 'd_1910',
 'd_1911',
 'd_1912',
 'd_1913']

In [22]:
WRMSSE_lgb = {}
pred_42840_df = convert_to_42840(preds1,valid_d_cols,groupbys)
    #Computed WRMSSE for each predictions based on hyper-parameters
WRMSSE_lgb[(125,0.075)] = custom_metric(train_42840_df,valid_42840_df,pred_42840_df,weight_df)


In [23]:
print('WRMSSE on test data using lgbmregressor is: ',WRMSSE_lgb)

WRMSSE on test data using lgbmregressor is:  {(125, 0.075): 0.6150332962599033}
